## Concurrencia

### Concurrencia vs paralelismo

 - Varias tareas vs una sola

 - Indeterminismo

In [42]:
import Control.Concurrent (forkIO)
import qualified Data.ByteString.Lazy as L
import Codec.Compression.GZip (compress)

compressFile = L.writeFile "files/foo.gz" . compress

do
    content <- L.readFile "files/foo.txt"
    forkIO (compressFile content)
    putStrLn "Gracias por comprimir!"
    return ()

Gracias por comprimir!

### Comunicación entre threads

Comunicación simple

Se utiliza Mvar - una variable sincronizable

Se puede ver como una caja con un valor

In [43]:
import Control.Concurrent

do
  m <- newEmptyMVar
  forkIO $ do
    v <- takeMVar m
    putStrLn ("Recibido " ++ show v)
  putStrLn "Enviando"
  putMVar m "Hola!"

Enviando
Recibido "Hola!"

Usar putMVar y takeMVar resulta en problemas comunes en concurrencia: deadlocks

modifyMVar es una combinación de los dos **segura**

Comunicación por canales

In [79]:
import Control.Concurrent
import Control.Concurrent.Chan

do
  ch <- newChan
  forkIO $ writeChan ch "Hola!"
  forkIO $ writeChan ch "Hola desde otro hilo!"
  readChan ch >>= print
  readChan ch >>= print

"Hola!"
"Hola desde otro hilo!"

### Paralelismo

Por defecto ghc usa un solo núcleo

Tenemos que pasar el parametro *-threaded* a la hora de linkear (creación del ejecutable)

Desventaja: mayor costo de creación de threads y manejo de MVars

In [78]:
import Control.Parallel (par)

parallelMap :: (a -> b) -> [a] -> [b]
parallelMap f (x:xs) = let r = f x
                       in r `par` r : parallelMap f xs
parallelMap _ _      = []

parallelMap (*2) [1,2,4,3,5,8,190]

[2,4,8,6,10,16,380]

*par* Evalua el valor del lado izquierdo y devuelve el del lado derecho. Hace la evaluación en paralelo.

## Software transactional memory

Mecanismo que trata de solucionar los problemas del modelo concurrente común. 

Un bloque de acciones se ejecuta como una transacción.
 - Si otros threads no modifican la misma data que ese bloque -> exito
 - Sino el bloque se descarta y se vuelve a ejecutar

*atomicaly* es el bloque. *STM* es una mónada parecida a IO (tiene efectos secundarios).

In [2]:
import Control.Concurrent.STM

:t atomically

atomically :: forall a. STM a -> IO a

*TVar* son contenedores mutables parecidos a MVar, pero solo se pueden usar en una acción STM.

In [25]:
type Account = TVar Int

withdraw :: Account -> Int -> STM ()
withdraw acc amount = do
    bal <- readTVar acc
    writeTVar acc (bal - amount)

deposit :: Account -> Int -> STM ()
deposit acc amount = withdraw acc (- amount)

atomically $ do 
    acc <- newTVar (20 :: Int)
    deposit acc 20
    withdraw acc 10
    readTVar acc


30

*Retry* permite reiniciar una transacción desde 0 (sin realizar todas las modificaciones)

In [24]:
limitedWithdraw :: Account -> Int -> STM ()
limitedWithdraw acc amount = do
    bal <- readTVar acc
    if amount > 0 && amount > bal
    then retry
    else writeTVar acc (bal - amount)

*orElse* permite hacer una acción en el caso de que otra falle (sea reiniciada)

In [23]:
limitedWithdraw2 :: Account -> Account -> Int -> STM ()
limitedWithdraw2 acc1 acc2 amt
  = orElse (limitedWithdraw acc1 amt) (limitedWithdraw acc2 amt)